In [1]:
!pip install tensorflow
!pip install pandas
!pip install matplotlib
!pip install opencv-python-headless


  Using cached https://files.pythonhosted.org/packages/91/68/c9ce4bd8989333cbb59b42c15d7a43e414d495dba3a21664411564c77b6a/tensorflow-2.6.2-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/30/98/baa62c0b771cb20b0381896fb61926d9bec783fb446eaa53efd4e5c70e3b/grpcio-1.48.2-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/6c/be/4e32d02bf08b8f76bf6e59f2a531690c1e4264530404501f3489ca975d9a/protobuf-4.21.0-py2.py3-none-any.whl


protobuf requires Python '>=3.7' but the running Python is 3.6.5
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/2f/7e/d20f68a5f1487adf19d74378d349932a386b1ece3be9be9915e5986db468/opencv-python-headless-4.10.0.84.tar.gz


  Missing build time requirements in pyproject.toml for opencv-python-headless from https://files.pythonhosted.org/packages/2f/7e/d20f68a5f1487adf19d74378d349932a386b1ece3be9be9915e5986db468/opencv-python-headless-4.10.0.84.tar.gz#sha256=f2017c6101d7c2ef8d7bc3b414c37ff7f54d64413a1847d89970b6b7069b4e1a: 'wheel'.
  This version of pip does not implement PEP 517 so it cannot build a wheel without 'setuptools' and 'wheel'.
  Could not find a version that satisfies the requirement numpy==1.21.0 (from versions: 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5)
No matching distribution found for numpy==1.21.0
You are using pip version 10.0.1, however version 21.3.1 is available.
You should consider upgrading via

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load annotations
annotations_path = 'Y:/NMML_Polar_Imagery_3/ForModelDevelopment_2024_ColorModel/BOSS_Color_Training/BOSS_TrainingImages_20220712_Detections.csv'
annotations = pd.read_csv(annotations_path)

# Load image filenames
filenames_path = 'Y:/NMML_Polar_Imagery_3/ForModelDevelopment_2024_ColorModel/BOSS_Color_Training/BOSS_TrainingImages_20220712_ImageList'
with open(filenames_path) as file:
    image_filenames = file.readlines()
    image_filenames = [line.strip() for line in image_filenames]


FileNotFoundError: [Errno 2] No such file or directory: 'Y:/NMML_Polar_Imagery_3/ForModelDevelopment_2024_ColorModel/BOSS_Color_Training/BOSS_TrainingImages_20220712_Detections.csv'

In [ ]:
# Directory where the images are stored
image_dir = 'Y:/NMML_Polar_Imagery_3/ForModelDevelopment_2024_ColorModel/BOSS_Color_Training'

# Function to load and preprocess ROIs
def extract_roi(image_path, bbox):
    """
    Extracts the region of interest (ROI) from the image based on bounding box coordinates.

    Args:
    - image_path (str): Path to the image file.
    - bbox (tuple): Bounding box coordinates (left, right, top, bottom).

    Returns:
    - roi (numpy array): Preprocessed ROI.
    """
    image = cv2.imread(image_path)
    if image is not None:
        x1, x2, y1, y2 = bbox
        roi = image[y1:y2, x1:x2]
        roi = cv2.resize(roi, (512, 512))  # Resize to a fixed size for the model
        roi = roi / 255.0  # Normalize to [0, 1]
        return roi
    else:
        return None

# Load images and labels
images = []
labels = []
original_images = []  # To keep the original images for visualization

for idx, row in annotations.iterrows():
    image_name = row['image_name']
    rgb_image_path = os.path.join(image_dir, image_name)
    if os.path.exists(rgb_image_path):
        bbox = (row['bound_left'], row['bound_right'], row['bound_top'], row['bound_bottom'])
        roi = extract_roi(rgb_image_path, bbox)
        if roi is not None:
            images.append(roi)
            labels.append(row['species_id'])
            original_images.append((rgb_image_path, bbox))

# Convert lists to arrays
images = np.array(images)
labels = np.array(labels)

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)

print(f"Loaded {len(images)} images.")


In [ ]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
train_datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

history = model.fit(train_generator, epochs=10, validation_data=val_generator)


In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation accuracy: {val_accuracy * 100:.2f}%")


In [ ]:
def plot_bbox(image_path, bbox):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    x1, x2, y1, y2 = bbox
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Visualize annotations on the last 50 images of the validation dataset
for i in range(-50, 0):
    plot_bbox(original_images[i][0], original_images[i][1])
